https://deap.readthedocs.io/en/master/examples/ga_onemax_numpy.html

In [1]:
import random
import numpy

from deap import base
from deap import creator
from deap import tools
from deap import algorithms

In [2]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", numpy.ndarray, fitness=creator.FitnessMax)

In [3]:
toolbox = base.Toolbox()
# Attribute generator 
toolbox.register("attr_bool", random.randint, 0, 1)
# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, 
    toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [4]:
def evalOneMax(individual):
    return sum(individual),

In [5]:
def cxTwoPointCopy(ind1, ind2):
    """Execute a two points crossover with copy on the input individuals. The
    copy is required because the slicing in numpy returns a view of the data,
    which leads to a self overwritting in the swap operation. It prevents
    ::
    
        >>> import numpy
        >>> a = numpy.array((1,2,3,4))
        >>> b = numpy.array((5,6,7,8))
        >>> a[1:3], b[1:3] = b[1:3], a[1:3]
        >>> print(a)
        [1 6 7 4]
        >>> print(b)
        [5 6 7 8]
    """
    size = len(ind1)
    cxpoint1 = random.randint(1, size)
    cxpoint2 = random.randint(1, size - 1)
    if cxpoint2 >= cxpoint1:
        cxpoint2 += 1
    else: # Swap the two cx points
        cxpoint1, cxpoint2 = cxpoint2, cxpoint1

    ind1[cxpoint1:cxpoint2], ind2[cxpoint1:cxpoint2] \
        = ind2[cxpoint1:cxpoint2].copy(), ind1[cxpoint1:cxpoint2].copy()
        
    return ind1, ind2

In [6]:
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", cxTwoPointCopy)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)


In [7]:
def main():
    pop = toolbox.population(n=300)
    hof = tools.HallOfFame(1, similar=numpy.array_equal)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2, ngen=40, 
                                   stats=stats, halloffame=hof, verbose=True)

In [8]:
main()

gen	nevals	avg  	std    	min	max
0  	300   	49.55	4.82571	34 	63 
1  	163   	53.1967	3.49638	43 	63 
2  	164   	55.76  	2.99818	48 	64 
3  	178   	58.1667	2.9267 	51 	66 
4  	177   	60.2233	3.05834	49 	68 
5  	170   	62.6933	2.61775	56 	70 
6  	193   	64.5867	2.53821	57 	73 
7  	178   	66.39  	2.60088	57 	75 
8  	176   	68.38  	2.50777	61 	79 
9  	164   	70.28  	2.82989	61 	79 
10 	193   	72.12  	2.98423	63 	80 
11 	198   	74.1767	3.14199	62 	82 
12 	184   	76.0633	2.5125 	67 	82 
13 	180   	77.5167	2.39368	67 	86 
14 	167   	78.8867	2.36513	70 	86 
15 	181   	80.0967	2.26877	73 	86 
16 	190   	81.2   	2.53509	71 	89 
17 	197   	82.6433	2.68504	69 	91 
18 	180   	83.9267	2.60665	75 	90 
19 	183   	85.3533	2.3098 	77 	91 
20 	184   	86.2967	2.47561	77 	92 
21 	183   	87.3667	2.30627	79 	92 
22 	191   	88.5133	2.24421	80 	94 
23 	155   	89.3933	2.26391	80 	95 
24 	170   	90.3033	2.249  	83 	96 
25 	185   	91.31  	2.28486	83 	95 
26 	179   	92.1633	2.20982	82 	97 
27 	165   	92.9767	2.236